In [1]:
import warnings
warnings.filterwarnings('ignore')

import ipywidgets as widgets
from IPython.display import display
from IPython.utils.traitlets import Unicode

def installPackages(sender):
    !pip install folium
    !pip install tweepy
install_btn = widgets.Button(description="Install Packages")
install_btn.on_click(installPackages) 
display(install_btn)

Button(description='Install Packages', style=ButtonStyle())

In [14]:
from pymongo import MongoClient

from IPython.utils.traitlets import Unicode

class PasswordWidget(widgets.Textarea):
    _view_name = Unicode('PasswordView', sync=True)


uri = PasswordWidget(placeholder='Input connection string of your MongoDB')
login_btn = widgets.Button(description="Login")

def login(sender):
    try:
        client = MongoClient(uri.value)
        global db
        db = client.twitterdb
        print('Connection established successfully')
    except ValueError:
        print(ValueError)

login_btn.on_click(login)

display(uri, login_btn)

PasswordWidget(value='', placeholder='Input connection string of your MongoDB')

Button(description='Login', style=ButtonStyle())

Connection established successfully


In [20]:
import tweepy

access_token = "917416602634768385-pXPkTeyW9vaysd4vZflYm2pZckkIeDn"
access_token_secret = "HweGKohJFWSMPDj1LwjoNExGIj1K2e7ApHdHpA7fcwl7F"
consumer_key = "r8dh3IsvxNOMH2UjSIoM00gnN"
consumer_secret = "0SqGVySPhZ8ngMYnQ05W8KPctMp8jbXGSdCo0qKxMYKwcewIrZ"

#access_token = "438291047-AWXl0LpNxZzjhdFA3FH7AJHtmLRK52QDJiKzq5Wz"
#access_token_secret = "o3kZKFF2s9ctgVpfDVRRpMbg6BMsGUIFWlJm9wSysKyyY"
#consumer_key = "q4utaFepGhE5OjujyoruBOoQg"
#consumer_secret = "D5K3P5URNUTxKnoVnggiUFsNapuNLOSx5cB7Zh6Y4HhpBhhtNy"

#consumer_key = 'cwuOhOSiMHaqSjUsyfYRVltuE'
#consumer_secret = 'JBZWaPi3ldDHgMo6NPr8MbRKEU2iHBW7xVzL094HjsoX33K4eJ'
#access_token = '842632842207203328-cNbwTaG4eW4rbQJwaG4RxtZkHJ51SoO'
#access_token_secret = 'IhypdlKWPYtpKJ8aWevWTPTyeTbtmffVRGsFcF9hXkQQg'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

print('twitter is connected')

twitter is connected


In [4]:
import json

class MyStreamListener(tweepy.StreamListener):

    def on_data(self, raw_data):
        json_data = json.loads(raw_data)
        post_id = db.twitter_data.insert_one(json_data).inserted_id
        print(post_id)
    def on_error(self, status_code):
        if status_code == 420:
            # returning False in on_data disconnects the stream
            return False
        
def streamDataFromGlasgow(sender):
    myStream = tweepy.Stream(auth = api.auth, listener=MyStreamListener())
    #myStream.filter(locations=[-4.50,55.79,-3.97,55.93], async=True)
stream_btn = widgets.Button(description="Stream Data")
stream_btn.on_click(streamDataFromGlasgow) 
display(stream_btn)

Button(description='Stream Data', style=ButtonStyle())

In [5]:
import folium
import re
import numpy as np
import math

def startMap(sender):
    m = folium.Map(location=[(55.79+55.93)/2, (-4.50-3.97)/2])
    filter = "is"
    regx = re.compile(".*"+filter+".*", re.IGNORECASE)
    posts = db.twitter_data.find({"text": regx})
    for post in posts:
            x0 = post["place"]["bounding_box"]["coordinates"][0][0][0]
            x1 = post["place"]["bounding_box"]["coordinates"][0][1][0]
            x2 = post["place"]["bounding_box"]["coordinates"][0][2][0]
            y0 = post["place"]["bounding_box"]["coordinates"][0][0][1]
            y1 = post["place"]["bounding_box"]["coordinates"][0][1][1]
            y2 = post["place"]["bounding_box"]["coordinates"][0][2][1]
            sq = np.square([x1-x0, y1-y0, x2-x1, y2-y1])
            sqrts = np.sqrt([sq[0]+sq[1], sq[2]+sq[3]])
            L1 = sqrts[0]
            L2 = sqrts[1]
            R = math.sqrt(L1**2 + L2**2)
            centre=[(y2+y0)/2, (x2+x0)/2]
            text = ''.join(e for e in post["text"] if e.isalnum() or e==' ')[:40]
            text = '<i>' + text + '</i>'
            if (R>1.0):
                folium.CircleMarker(centre, radius=R, popup=text,
                                color='#3186cc', fill_color='#3186cc').add_to(m)
            else:
                folium.Marker(centre, popup=text,
                             icon=folium.Icon(color='green',icon='info-sign')).add_to(m)
    display(m)
                
start_btn = widgets.Button(description="Start map")  
start_btn.on_click(startMap)

display(start_btn)

Button(description='Start map', style=ButtonStyle())

In [22]:
def decodeData(status):
    return {
        "retweet_count": status.retweet_count,
        "favorited": status.favorited,
        "in_reply_to_user_id": status.in_reply_to_user_id,
        "created_at": status.created_at,
        "coordinates": status.coordinates,
        "user": {
            "created_at": status.user.created_at,
            "geo_enabled": status.user.geo_enabled,
            "lang": status.user.lang,
            "url": status.user.url,
            "description": status.user.description,
            "time_zone": status.user.time_zone,
            "location": status.user.location,
            "screen_name": status.user.screen_name,
            "protected": status.user.protected,
            "statuses_count": status.user.statuses_count,
            "profile_image_url_https": status.user.profile_image_url_https,
            "utc_offset": status.user.utc_offset,
            "followers_count": status.user.followers_count,
            "id": status.user.id,
            "id_str": status.user.id_str,
            "name": status.user.name,
            "friends_count": status.user.friends_count,
        },
        "retweeted": status.user.retweeted,
        "geo": status.user.geo,
        "id": status.id,
        "text": status.text,
        "lang": status.lang,
        "in_reply_to_user_id_str": status.in_reply_to_user_id_str,
        "id_str": status.id_str
    }

def getHistoricalStatuses(sender):
    statuses = list(tweepy.Cursor(api.search,
        q="is",
        geocode="55.85,-4.25,10km",
        since="2017-10-20",
        #until="2017-10-21",
        lang="en").pages())

    geo_enabled_statuses = [status for status in statuses[0] if status.place!=None]
    db.twitter_historical_data.update(geo_enabled_statuses, geo_enabled_statuses, upsert = True)
    
history_btn = widgets.Button(description="Take Data")  
history_btn.on_click(getHistoricalStatuses)

display(history_btn)

Button(description='Take Data', style=ButtonStyle())